
<br>
<font>
<div dir=ltr align=center>
<img src="https://cdn.freebiesupply.com/logos/large/2x/sharif-logo-png-transparent.png" width=150 height=150> <br>
<font color=0F5298 size=7>
Artificial Intelligence <br>
<font color=2565AE size=5>
Computer Engineering Department <br>
Spring 2024<br>
<font color=3C99D size=5>
Practical Assignment 5 - RL <br>
<font color=696880 size=4>
Amirhossein Abedi


____

# Personal Data

In [1]:
# Set your student number and name
student_number = '401106339'
Name = 'Ali'
Last_Name = 'Ghasemzadeh'

# <center>تاکسی</center>

<div dir="rtl" style="font-size:18px">
در این سوال می‌خواهیم با محیط Taxi در gymnasium کار کنیم. 
<br>
در اینجا در نقش یک تاکسی هستیم که باید یک مسافر را از یک مکان مشخص برداریم و در مقصد موردنظر او پیاده کنیم. توضیحات تکمیلی درمورد این محیط را می‌توانید در  <a href="https://gymnasium.farama.org/environments/toy_text/taxi/">Taxi</a>  پیدا کنید.

![Taxi](taxi.png)
</div>

<div dir="rtl" style="font-size:18px">
در این سوال می‌خواهیم با پیاده‌سازی الگوریتم value iteration بهترین پاسخ در هر حالت را پیدا کنیم. هر حالت در این بازی با یک عدد بین ۰ تا ۴۹۹ نشان داده می‌شود. که فرمول آن به صورت زیر است
<br><br>

### <center dir="ltr">state = ((5 × row + col) × 5 + passenger_location) × 4 + destination</center>

<br><br>
مقادیر row و col که مشخص است و عددی بین ۰ تا ۴ است. مقدار passenger_location بین ۰ تا ۴ می‌باشد که هرکدام از اعداد نشان می‌دهد که مسافر در کدام پایانه قرار دارد. پایانه‌ها مکان‌هایی مشخص درون بازی هستند که می‌تواند با استفاده از تابع env.render آن‌ها را مشاهده کنید. ۴ پایانه به رنگ‌های قرمز، سبز، زرد، آبی درون بازی قرار دارند که مقادیر ۰ تا ۳ به ترتیب همین پایانه‌ها را نشان می‌دهد. و در نهایت مقدار ۴ برای passenger_location به این معنا است که مسافر در تاکسی قرار دارد.
<br><br>
مقدار destination نشان می‌دهد که مسافر می‌خواهد در کدام پایانه پیاده شود. نگاشت عدد و رنگ‌ها مانند بخش passenger_location می‌باشد و فقط مقادیر ۰ تا ۳ را به خود می‌گیرد.
<br><br>
با هر بار اجرای تابع reset و step بر روی env ، مقدار اولی که به شما برگردانده می‌شود مقدار state است. در ادامه به action ها می‌پردازیم. در کل ۶ اکشن مختلف می‌توانیم اجرا کنیم. اکشن ۰ تا ۳ به ترتیب مربوط به حرکات پایین، بالا، راست و چپ است. حرکت ۴ مربوط به برداشتن مسافر و حرکت ۵ پیاده کردن مسافر است. استفاده از حرکت ۴ و ۵ هنگامی که مجاز نباشند، ریوارد منفی زیادی خواهد داشت.
</div>

In [2]:
import gymnasium as gym
env = gym.make('Taxi-v3', render_mode='ansi')

<div dir="rtl" style="font-size:18px">
خروجی تابع reset شامل یک ۲ تایی است. اولین خروجی آن یک عدد است که نشان‌دهنده حالت اولیه می‌باشد و خروجی دوم یک دیکشنری است که مقدار action_mask شامل تمام اکشن‌هایی است که در این حالت می‌توانید انجام دهید.
</div>

In [3]:
env.reset()

(249, {'prob': 1.0, 'action_mask': array([1, 1, 1, 1, 0, 0], dtype=int8)})

<div dir="rtl" style="font-size:18px">
خروجی تابع step هم تا حدود زیادی شبیه به reset می باشد. با این تفاوت که بعد از خروجی حالت، یک عدد و ۲ بولین به شما می‌دهد که عدد اول مقدار reward و بولین اول تمام شدن بازی را مشخص می‌کند.
</div>

In [4]:
env.step(0)

(349,
 -1,
 False,
 False,
 {'prob': 1.0, 'action_mask': array([1, 1, 0, 1, 0, 0], dtype=int8)})

<div dir="rtl" style="font-size:18px">
در ادامه کلاس VI را پیاده‌سازی کنید که با گرفتن discount factor الگوریتم value iteration را پیاده‌سازی می‌کند.
</div>

In [5]:
import numpy as np

class VI():
    def __init__(self, gamma: float = 0.95):
        self.env = env
        self.gamma = gamma
        self.theta = 1e-6
        self.state_size = self.env.observation_space.n 
        self.action_size = self.env.action_space.n
        self.V = np.zeros(self.state_size)
        self.policy_table = np.zeros(self.state_size, dtype=int)
        self.value_iteration()

    def value_iteration(self):
        while True:
            delta = 0
            for s in range(self.state_size):
                v = self.V[s]
                q_values = np.zeros(self.action_size)
                for a in range(self.action_size):
                    transition = self.env.unwrapped.P[s][a]
                    for prob, next_state, reward, done in transition:
                        q_values[a] += prob * (reward + self.gamma * self.V[next_state])
                self.V[s] = np.max(q_values)
                delta = max(delta, np.abs(v - self.V[s]))
            if delta < self.theta:
                break
        
        # Extracting optimal policy
        for s in range(self.state_size):
            q_values = np.zeros(self.action_size)
            for a in range(self.action_size):
                transition = self.env.unwrapped.P[s][a]
                for prob, next_state, reward, done in transition:
                    q_values[a] += prob * (reward + self.gamma * self.V[next_state])
            self.policy_table[s] = np.argmax(q_values)


    def policy(self, state: int) -> int:
        return self.policy_table[state]

    def _num_to_state(self, num: int):
        dest = num % 4
        num //= 4
        passenger_loc = num % 5
        num //= 5
        col = num % 5
        num //= 5
        row = num
        return row, col, passenger_loc, dest

    def _state_to_num(self, row, col, passenger_loc, dest):
        ret = row
        ret = 5 * ret + col
        ret = 5 * ret + passenger_loc
        ret = 4 * ret + dest
        return ret
    # positions of R,G,Y,B
    def decode_location(self, location):
        return [(0, 0), (0, 4), (4, 0), (4, 3)][location]

agent = VI()

<div dir="rtl" style="font-size:18px">
۲ سلول بعدی را اجرا کنید. در هر نقشه نشان داده شده است که با برقرار بودن شرایط به خصوص، تاکسی به کدام سمت حرکت می‌کند. در انتها در چند خط منطقی عمل کردن مدل خود را بررسی کنید.
</div>

In [6]:
terminal = {0: 'R', 1: 'G', 2: 'Y', 3: 'B'}
arrows = {0: '↓', 1: '↑', 2: '→', 3: '←'}
for passenger_loc in range(4):
    print(f'Passenger in {terminal[passenger_loc]}')
    print('+---------+')
    for i in range(5):
        print('|', end='')
        for j in range(5):
            if not j == 0:
                if (i, j) in [(0, 2), (1, 2), (3, 1), (4, 1), (3, 3), (4, 3)]:
                    print('|', end='')
                else:
                    print(':', end='')
            if (i, j) in [(0, 0), (0, 4), (4, 0), (4, 3)]:
                print(terminal[[(0, 0), (0, 4), (4, 0), (4, 3)].index((i, j))], end='')
            else:
                try:
                    print(arrows[agent.policy(agent._state_to_num(i, j, passenger_loc, 1))], end='')
                except:
                    print('⤬', end='')
        print('|')
    print('+---------+\n')    

Passenger in R
+---------+
|R:←|↓:↓:G|
|↑:↑|↓:↓:↓|
|↑:↑:←:←:←|
|↑|↑:↑|↑:↑|
|Y|↑:↑|B:↑|
+---------+

Passenger in G
+---------+
|R:↓|→:→:G|
|↓:↓|→:→:↑|
|→:→:→:→:↑|
|↑|→:↑|→:↑|
|Y|→:↑|B:↑|
+---------+

Passenger in Y
+---------+
|R:↓|↓:↓:G|
|↓:↓|↓:↓:↓|
|↓:←:←:←:←|
|↓|↑:↑|↑:↑|
|Y|↑:↑|B:↑|
+---------+

Passenger in B
+---------+
|R:↓|↓:↓:G|
|↓:↓|↓:↓:↓|
|→:→:→:↓:↓|
|↑|→:↑|↓:↓|
|Y|→:↑|B:←|
+---------+



In [7]:
terminal = {0: 'R', 1: 'G', 2: 'Y', 3: 'B'}
arrows = {0: '↓', 1: '↑', 2: '→', 3: '←'}
for dest in range(4):
    print(f'Passenger on board, destination in {terminal[dest]}')
    print('+---------+')
    for i in range(5):
        print('|', end='')
        for j in range(5):
            if not j == 0:
                if (i, j) in [(0, 2), (1, 2), (3, 1), (4, 1), (3, 3), (4, 3)]:
                    print('|', end='')
                else:
                    print(':', end='')
            if (i, j) in [(0, 0), (0, 4), (4, 0), (4, 3)]:
                print(terminal[[(0, 0), (0, 4), (4, 0), (4, 3)].index((i, j))], end='')
            else:
                try:
                    print(arrows[agent.policy(agent._state_to_num(i, j, 4, dest))], end='')
                except:
                    print('⤬', end='')
        print('|')
    print('+---------+\n')

Passenger on board, destination in R
+---------+
|R:←|↓:↓:G|
|↑:↑|↓:↓:↓|
|↑:↑:←:←:←|
|↑|↑:↑|↑:↑|
|Y|↑:↑|B:↑|
+---------+

Passenger on board, destination in G
+---------+
|R:↓|→:→:G|
|↓:↓|→:→:↑|
|→:→:→:→:↑|
|↑|→:↑|→:↑|
|Y|→:↑|B:↑|
+---------+

Passenger on board, destination in Y
+---------+
|R:↓|↓:↓:G|
|↓:↓|↓:↓:↓|
|↓:←:←:←:←|
|↓|↑:↑|↑:↑|
|Y|↑:↑|B:↑|
+---------+

Passenger on board, destination in B
+---------+
|R:↓|↓:↓:G|
|↓:↓|↓:↓:↓|
|→:→:→:↓:↓|
|↑|→:↑|↓:↓|
|Y|→:↑|B:←|
+---------+



<p dir='rtl'>
عملکرد کدمان درست است زیرا با کمترین تعداد حرکت به مبدا و مقصد می رسیم
</p>